In [1]:
!git clone https://github.com/mehdi-mirzapour/RezoJDM-SDS.git

Cloning into 'RezoJDM-SDS'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 106 (delta 57), reused 74 (delta 28), pack-reused 0
Receiving objects: 100% (106/106), 1.44 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd RezoJDM-SDS/

/content/RezoJDM-SDS


In [49]:
!mkdir 'OpenKE format'

In [39]:
import pandas as pd
import numpy as np
train = pd.read_csv('datasets/train.csv')
test  = pd.read_csv('datasets/test.csv')

## Statistics of Dataset
In this part we want to know how many nodes and relations our graph has
### Output: # relation and # nodes

In [40]:
Relation2id = {j:i for i,j in enumerate(train.keys()[2:])}
Id2Relation = {j:i for i , j in Relation2id.items()}
Entities = list(set(train['source_name'].values.tolist() + train['destination_name'].values.tolist() + test['source_name'].values.tolist() + test['destination_name'].values.tolist()))
Entities2id = {j:i for i,j in enumerate(Entities)}
len(Relation2id),len(Entities)

(10, 23654)

## Dataset to triples: paris france 01000000 => paris capitalof paris
For using some embedding libaries such as OpenKE we need to transfor our dataset to triples : (head, tail, relation)

In [41]:
def csv2triple(train):
  Train_triples = []
  for row in train.values.tolist():
    head = row[0]
    tail = row[1]
    for i in range(len(row[2:])):
      if row[2+i] == 1:
        relation = Id2Relation[i]
        Train_triples.append([head,tail,relation])
  return Train_triples

Train_triples = csv2triple(train)
Test_triples  = csv2triple(test)
len(Train_triples),len(Test_triples)

(104030, 26032)

In [42]:
Train_triples[:10]

[['espagnol', 'Espagne', 'r_holo'],
 ['Astéracées', 'marguerite', 'r_has_part'],
 ['pleurer', 'trauma', 'r_causatif'],
 ['Manet', 'artiste-peintre', 'r_isa'],
 ['mouvement', 'ordre', 'r_causatif'],
 ['cétacé', 'mer', 'r_lieu'],
 ['marin', 'voilier', 'r_holo'],
 ['marin', 'voilier', 'r_lieu'],
 ['CV', 'biographie', 'r_isa'],
 ['rouler', 'homme', 'r_patient']]

## Now we should convert Dataset to numerics

In [43]:
def Triples2IDs(triples):
  for i in range(len(triples)):
    triples[i][0] = Entities2id[triples[i][0]]
    triples[i][1] = Entities2id[triples[i][1]]
    triples[i][2] = Relation2id[triples[i][2]]
  return triples    
train = Triples2IDs(Train_triples)
test  = Triples2IDs(Test_triples)

## Conver to files

In [47]:
train[:5]

[[16115, 18634, 5],
 [10123, 968, 4],
 [5564, 10624, 2],
 [13971, 2530, 7],
 [2413, 19727, 2]]

In [54]:
def graph2file(triples,file_name):
  file = open('OpenKE format/' + str(file_name),'w')
  file.write(str(len(triples))+ '\n')
  for i in range(len(triples)):
    file.write(str(triples[i][0]) +' '+ str(triples[i][1]) +' '+ str(triples[i][2])+'\n')
  file.close()
graph2file(train,'train2id.txt')
graph2file(test,'test2id.txt')
graph2file(test,'valid2id.txt') # the same BC we need it

In [55]:
file = open('OpenKE format/relation2id.txt','w')
file.write(str(len(Id2Relation))+ '\n')
for k,v in Id2Relation.items():
  file.write(str(k) +'\t'+ str(v)+'\n')
file.close()

In [57]:
file = open('OpenKE format/entity2id.txt','w')
file.write(str(len(Entities2id))+ '\n')
for k,v in Entities2id.items():
  file.write(str(k) +'\t'+ str(v)+'\n')
file.close()

In [63]:
!zip -r 'opke.zip' 'OpenKE format'

updating: OpenKE format/ (stored 0%)
  adding: OpenKE format/test2id.txt (deflated 61%)
  adding: OpenKE format/valid2id.txt (deflated 61%)
  adding: OpenKE format/train2id.txt (deflated 61%)
  adding: OpenKE format/relation2id.txt (deflated 24%)
  adding: OpenKE format/entity2id.txt (deflated 52%)
